## Global-based Inverse Design
---

## Import Module

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import pickle
import pymiecs
import tensorflow as tf
from tensorflow import keras
import nevergrad as ng
from concurrent import futures
import time

2025-02-27 15:30:45.543212: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-27 15:30:45.551224: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-27 15:30:45.560137: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-27 15:30:45.562694: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 15:30:45.569482: I tensorflow/core/platform/cpu_feature_guar

## Reload the data scaler

In [2]:
# preprocessor path
preprocessor_path = "datasets/scaler_particle_geometries.pkl"
scaler_Qfwd_path = "datasets/scaler_Qfwd.pkl"
scaler_Qback_path = "datasets/scaler_Qback.pkl"

# Load the preprocessors and scalers
with open(preprocessor_path, "rb") as f:
    preprocessor = pickle.load(f)
with open(scaler_Qfwd_path, "rb") as f:
    scaler_Qfwd = pickle.load(f)
with open(scaler_Qback_path, "rb") as f:
    scaler_Qback = pickle.load(f)

### Mie Utils

In [3]:
# %% --- Mie
def get_Mie_spec(wavelengths, r_core, r_shell, mat_core, mat_shell, n_env):

    k0 = 2 * np.pi / wavelengths
    n_core = mat_core.get_refindex(wavelengths)
    n_shell = mat_shell.get_refindex(wavelengths)

    res = pymiecs.Q(
        k0,
        r_core=r_core,
        n_core=n_core,
        r_shell=r_shell,
        n_shell=n_shell,
        n_env=n_env.real**0.5,  # host medium must be lossless
    )
    return (
        res["qsca"],
        res["qback"],
        res["qfwd"],
    )

In [4]:
Si = pymiecs.materials.MaterialDatabase("Si")
SiO2 = pymiecs.materials.MaterialDatabase("SiO2")
Si3N4 = pymiecs.materials.MaterialDatabase("Si3N4")
Au = pymiecs.materials.MaterialDatabase("Au")
Ag = pymiecs.materials.MaterialDatabase("Ag")
ZrO2 = pymiecs.materials.MaterialDatabase("ZrO2")
TiO2 = pymiecs.materials.MaterialDatabase("TiO2")


# Define a function to map material names to material objects
def get_material(material_name):
    if material_name == "Si":
        return Si
    elif material_name == "SiO2":
        return SiO2
    elif material_name == "Au":
        return Au
    elif material_name == "Ag":
        return Ag
    elif material_name == "Si3N4":
        return Si3N4
    elif material_name == "ZrO2":
        return ZrO2
    elif material_name == "TiO2":
        return TiO2
    else:
        raise ValueError(f"Unknown material: {material_name}")

## reload Test data 

In [5]:
hdf5_df_file = "datasets/core_shell_particles_raw_122500_test_with_pred.h5"  # depend on what test set that forward model evalaute on
df_test = pd.read_hdf(hdf5_df_file)
df_test.head()  # 2500 sam

,mat_core,mat_shell,r_core,r_shell,wavelength,Q_sca,Q_back,Q_fwd,Q_fwd_pred,Q_back_pred
0,ZrO2,Si,39,132,"[400.0, 406.3492063492063, 412.6984126984127, ...","[2.038010369711505, 1.8095124021006057, 1.4807...","[0.6439440129016081, 0.44670942047839124, 0.55...","[8.631606079996017, 7.795760005483267, 6.22831...","[8.785859, 8.286023, 6.1634374, 4.3949394, 4.2...","[0.81082445, 0.509972, 0.4711062, 0.62237597, ..."
1,Au,Si3N4,65,92,"[400.0, 406.3492063492063, 412.6984126984127, ...","[0.7478421540587286, 0.7347653970480106, 0.723...","[0.25254436594760143, 0.2641361829295818, 0.27...","[3.0595320722390023, 2.910348641419738, 2.7718...","[2.9980655, 2.9284086, 2.8653688, 2.730851, 2....","[0.33432025, 0.30205694, 0.30620533, 0.3165547..."
2,Si,Si,31,32,"[400.0, 406.3492063492063, 412.6984126984127, ...","[0.42059673950184884, 0.2840222012565064, 0.21...","[0.12245274773436957, 0.03833342713719667, 0.0...","[1.1619876883360172, 0.8312774845838197, 0.618...","[0.92039216, 1.3765805, 1.5048053, 1.2357119, ...","[1.2127657, 0.83550537, 0.32235485, 0.0964607,..."
3,ZrO2,ZrO2,81,114,"[400.0, 406.3492063492063, 412.6984126984127, ...","[4.515870402326752, 4.215809011163139, 4.05102...","[2.008253901402069, 1.6064395105460934, 1.4057...","[19.18042348996285, 16.158873919994733, 14.280...","[18.862698, 16.10313, 14.292469, 12.995516, 11...","[1.9835316, 1.5604014, 1.3563392, 1.3436563, 1..."
4,Au,TiO2,49,76,"[400.0, 406.3492063492063, 412.6984126984127, ...","[2.5633444541178934, 2.5892776814208465, 2.588...","[1.6053755830270218, 1.4718419539005771, 1.314...","[6.575478822430851, 6.834712459103053, 7.04137...","[6.3441553, 6.6466975, 7.2830606, 7.7183237, 7...","[2.265903, 2.1518085, 2.0174856, 1.8726242, 1...."


In [7]:
df_test.describe()

,r_core,r_shell
count,2500.000000,2500.000000
mean,49.083600,100.342000
std,28.653925,40.494387
min,1.000000,6.000000
25%,25.000000,72.000000
50%,49.000000,100.500000
75%,73.000000,130.000000
max,100.000000,200.000000


In [8]:
df_test["log_Qfwd"] = df_test["Q_fwd"].apply(lambda x: np.log1p(np.array(x)))
df_test["log_Qback"] = df_test["Q_back"].apply(lambda x: np.log1p(np.array(x)))

## Objective Function

In [9]:
def calculate_mie_scattering_error(
    r_core,
    r_shell,
    core_material,
    shell_material,
    target_Qfwd_transformed,
    target_Qback_transformed,
):
    # Define wavelengths
    wavelengths = np.linspace(400, 800, 64)

    # Get material properties using your material function
    mat_core = get_material(core_material)
    mat_shell = get_material(shell_material)

    # Calculate Mie scattering using the new package
    Qsca, Qback, Qfwd = get_Mie_spec(
        wavelengths=wavelengths,
        r_core=r_core,
        r_shell=r_shell,
        mat_core=mat_core,
        mat_shell=mat_shell,
        n_env=1.0,  # Medium refractive index
    )

    # Apply log1p transformation to Mie Qfwd and Qback
    log_mie_Qfwd = np.log1p(Qfwd)
    log_mie_Qback = np.log1p(Qback)

    # Now, apply MinMax scaling to log-transformed Mie values
    log_mie_Qfwd_transformed = scaler_Qfwd.transform(log_mie_Qfwd.reshape(1, -1))
    log_mie_Qback_transformed = scaler_Qback.transform(log_mie_Qback.reshape(1, -1))

    # Compute the MSE between the predicted and target values
    mse_Qfwd = np.mean((log_mie_Qfwd_transformed - target_Qfwd_transformed) ** 2)
    mse_Qback = np.mean((log_mie_Qback_transformed - target_Qback_transformed) ** 2)

    # Total error is the sum of MSEs
    total_error = mse_Qfwd + mse_Qback
    return total_error

In [10]:
def objective_function(
    geometry_params, target_Qfwd_transformed, target_Qback_transformed
):
    r_core_real, r_shell_real, mat_core, mat_shell = geometry_params
    total_error = calculate_mie_scattering_error(
        r_core_real,
        r_shell_real,
        mat_core,
        mat_shell,
        target_Qfwd_transformed,
        target_Qback_transformed,
    )

    return total_error

## Run global Optimization

In [11]:
# Define the search space for core radius, shell thickness, core material, shell material
parametrization = ng.p.Tuple(
    ng.p.Scalar(lower=1, upper=100),  # Real core radius (nm)
    ng.p.Scalar(lower=6, upper=200),  # Real shell thickness (nm)
    ng.p.Choice(["Si", "SiO2", "Au", "Ag", "Si3N4", "ZrO2", "TiO2"]),  # Core material
    ng.p.Choice(["Si", "SiO2", "Au", "Ag", "Si3N4", "ZrO2", "TiO2"]),  # Shell material
)

In [15]:
# Initialize lists to store results and runtimes
sample_runtimes = []
results = []
num_workers = 25
budget = 1000

for idx, row in df_test.iterrows():
    print(f"Processing sample {idx + 1}/{len(df_test)}")

    # Extract the target data for this sample
    target_Qfwd = np.array(row["log_Qfwd"]).reshape(1, -1)
    target_Qback = np.array(row["log_Qback"]).reshape(1, -1)

    # Apply MinMax scaling to the target
    target_Qfwd_transformed = scaler_Qfwd.transform(target_Qfwd)
    target_Qback_transformed = scaler_Qback.transform(target_Qback)
    obj_fun = lambda params: objective_function(
        params, target_Qfwd_transformed, target_Qback_transformed
    )

    # Initialize Differential Evolution optimizer with specified crossover and population size
    optim_algo = ng.families.DifferentialEvolution(
        crossover="twopoints", popsize=num_workers
    )

    optimizer = optim_algo(parametrization, budget=budget)
    start_time = time.time()

    # Use a ThreadPoolExecutor for parallel batch processing
    with futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        recommendation = optimizer.minimize(obj_fun, executor=executor)

    end_time = time.time()
    sample_runtime = end_time - start_time

    # Append runtime and best geometry to the results
    sample_runtimes.append({"Sample": idx + 1, "Runtime (seconds)": sample_runtime})
    print(f"Runtime for sample {idx}: {sample_runtime:.2f} seconds")

    best_geometry = recommendation.value
    best_loss = recommendation.loss
    print(f"Best geometry for sample {idx}: {best_geometry}")
    print(f"Best loss for sample {idx}: {best_loss}")
    print("-------------------------------------------------------")

    # Append results to the list
    result = {
        "Sample Index": idx + 1,
        "Best Geometry": best_geometry,
        "Loss": best_loss,
        "Runtime (seconds)": sample_runtime,
    }

    results.append(result)

"""
##########################################################################################
####################### Convert the runtimes into a DataFrame ############################
##########################################################################################
"""
# Convert the results and runtimes to DataFrames and save
runtime_df = pd.DataFrame(sample_runtimes)
best_df = pd.DataFrame(results)

sample_runtime_path = "runtime/global_inverse_runtime.pkl"

with open(sample_runtime_path, "wb") as f:
    pickle.dump(runtime_df, f)

Processing sample 1/2500
Runtime for sample 0: 4.34 seconds
Best geometry for sample 0: (77.33785802915365, 103.49947848488426, 'ZrO2', 'Au')
Best loss for sample 0: 0.13026948074144137
-------------------------------------------------------
Processing sample 2/2500
Runtime for sample 1: 4.32 seconds
Best geometry for sample 1: (65.44400243575352, 84.98011066816302, 'SiO2', 'Au')
Best loss for sample 1: 0.005361681876081288
-------------------------------------------------------
Processing sample 3/2500
Runtime for sample 2: 3.21 seconds
Best geometry for sample 2: (48.419073136139936, 31.936499032634522, 'Si', 'Si')
Best loss for sample 2: 1.3728779609324832e-06
-------------------------------------------------------
Processing sample 4/2500
Runtime for sample 3: 4.15 seconds
Best geometry for sample 3: (93.3560948756604, 117.46363004106246, 'ZrO2', 'Si3N4')
Best loss for sample 3: 0.002124479415610161
-------------------------------------------------------
Processing sample 5/2500
Ru

In [ ]:
# Split the 'Best Geometry' column into multiple columns
best_df[["r_core", "r_shell", "mat_core", "mat_shell"]] = pd.DataFrame(
    best_df["Best Geometry"].tolist(), index=best_df.index
)

# Drop the 'Best Geometry' column if you no longer need it
best_df = best_df.drop(columns=["Best Geometry"])
best_df = best_df[
    [
        "mat_core",
        "mat_shell",
        "r_core",
        "r_shell",
        "Sample Index",
        "Runtime (seconds)",
        "Loss",
    ]
]

In [19]:
best_df.head()

,mat_core,mat_shell,r_core,r_shell,Sample Index,Runtime (seconds),Loss
0,ZrO2,Au,77.337858,103.499478,1,4.338133,0.130269
1,SiO2,Au,65.444002,84.980111,2,4.323697,0.005362
2,Si,Si,48.419073,31.936499,3,3.205805,0.000001
3,ZrO2,Si3N4,93.356095,117.463630,4,4.147974,0.002124
4,Au,TiO2,49.023123,77.253263,5,4.145138,0.008713


## use Mie to calculate Qfwd and Qback

In [20]:
wavelengths = np.linspace(400, 800, 64)  # From 400 nm to 800 nm
n_env = 1.0
mie_Qfwd_list, mie_Qback_list = [], []


for idx, row in best_df.iterrows():
    mat_core = get_material(row["mat_core"])
    mat_shell = get_material(row["mat_shell"])
    r_core = row["r_core"]
    r_shell = row["r_shell"]

    # Calculate the Mie spectrum for this geometry
    _, Qback, Qfwd = get_Mie_spec(
        wavelengths, r_core, r_shell, mat_core, mat_shell, n_env
    )

    mie_Qfwd_list.append(Qfwd)
    mie_Qback_list.append(Qback)

# Convert the Mie Qfwd and Qback to DataFrame or arrays if needed
mie_Qfwd_array = np.array(mie_Qfwd_list)
mie_Qback_array = np.array(mie_Qback_list)

# Validation: Check dimensions
assert len(mie_Qfwd_array) == len(best_df), "Mismatch in dimensions for Qfwd!"
assert len(mie_Qback_array) == len(best_df), "Mismatch in dimensions for Qback!"

# Add the Mie results to the DataFrame
best_df["mie_Qfwd"] = mie_Qfwd_array.tolist()
best_df["mie_Qback"] = mie_Qback_array.tolist()

best_df

,mat_core,mat_shell,r_core,r_shell,Sample Index,Runtime (seconds),Loss,mie_Qfwd,mie_Qback
0,ZrO2,Au,77.337858,103.499478,1,4.338133,1.302695e-01,"[7.943693523704477, 7.631676005310999, 7.35065...","[0.12938060934056017, 0.20866880941269564, 0.2..."
1,SiO2,Au,65.444002,84.980111,2,4.323697,5.361682e-03,"[2.6381263938781765, 2.5292054290052843, 2.427...","[0.33209261045765476, 0.3435998575540038, 0.35..."
2,Si,Si,48.419073,31.936499,3,3.205805,1.372878e-06,"[1.137715927515656, 0.8148088614955376, 0.6078...","[0.11457456575332535, 0.03793865784512998, 0.0..."
3,ZrO2,Si3N4,93.356095,117.463630,4,4.147974,2.124479e-03,"[20.180305365698967, 17.00894306872726, 14.976...","[1.502571295840774, 1.187915005201223, 1.03181..."
4,Au,TiO2,49.023123,77.253263,5,4.145138,8.713461e-03,"[6.220173089120092, 6.539505428696962, 6.85150...","[1.9077666986221749, 1.8125993548908415, 1.691..."
...,...,...,...,...,...,...,...,...,...
95,Si,SiO2,12.655562,110.773728,96,3.522580,1.324429e-04,"[4.887071450682193, 4.537841760867851, 4.21652...","[0.042395122311326125, 0.0436775509590038, 0.0..."
96,SiO2,SiO2,64.108087,139.124080,97,3.427480,1.692611e-06,"[10.926662000882738, 10.406217146938673, 9.920...","[0.42661206160909865, 0.3967152141798502, 0.36..."
97,Ag,Au,83.799607,89.945023,98,4.310959,1.150179e-02,"[9.226605258327384, 8.796238415844698, 8.34922...","[0.34805198953304517, 0.5533518381048385, 0.79..."
98,Au,Au,3.915361,146.000265,99,5.208300,2.164100e-11,"[13.193617466151618, 12.835768097657612, 12.50...","[0.8633008563057076, 0.9085297409362679, 0.939..."


### Also compare Mie to foward model predictions

## Reload the forward model

### Define the own resblock class

keras requires custom classes to be defined for being able to reload

In [21]:
@keras.utils.register_keras_serializable()
class ResBlock1D(keras.Model):
    def __init__(self, filters, kernel_size=3, convblock=False, **kwargs):
        super(ResBlock1D, self).__init__(**kwargs)

        # setup all necessary layers
        self.conv1 = keras.layers.Conv1D(filters, kernel_size, padding="same")
        self.bn1 = keras.layers.BatchNormalization()

        self.conv2 = keras.layers.Conv1D(filters, kernel_size, padding="same")
        self.bn2 = keras.layers.BatchNormalization()

        self.relu = keras.layers.LeakyReLU(negative_slope=0.1)

        self.convblock = convblock
        if self.convblock:
            self.conv_shortcut = keras.layers.Conv1D(filters, 1)

    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training=training)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)

        # add shortcut. optionally pass it through a Conv
        if self.convblock:
            x_sc = self.conv_shortcut(input_tensor)
        else:
            x_sc = input_tensor
        x += x_sc
        return self.relu(x)

    def get_config(self):
        base_config = super().get_config()
        return {
            "filters": self.filters,
            "kernel_size": self.kernel_size,
            "convblock": self.convblock,
            **base_config,
        }

In [23]:
forward_path = "models/resnet_Mie_predictor.keras"
forward_model = keras.models.load_model(forward_path)

I0000 00:00:1740667473.107556   28373 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740667473.130864   28373 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740667473.133120   28373 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740667473.135556   28373 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [24]:
"""
#########################################################################################
############### Use forward model to predict on the optimized geometries  ###############
#########################################################################################
"""

# Ensure that the columns are ordered the same way as the input during training
categorical_features = [
    "mat_core",
    "mat_shell",
]  # Update these names based on your data
numerical_features = ["r_core", "r_shell"]

# Use the preprocessor to transform the data
X_optimized_preprocessed = preprocessor.transform(
    best_df[categorical_features + numerical_features]
)

# Predict using the forward model
y_pred = forward_model.predict(X_optimized_preprocessed)

# Separate predictions for Qfwd and Qback
y_pred_Qfwd = y_pred[..., 0]  # First output for Qfwd predictions
y_pred_Qback = y_pred[..., 1]  # Second output for Qback predictions

# Inverse transform the predicted Qfwd and Qback to get the original scale
predicted_Qfwd_original = scaler_Qfwd.inverse_transform(y_pred_Qfwd)
predicted_Qback_original = scaler_Qback.inverse_transform(y_pred_Qback)

# Apply expm1 to reverse the log1p transformation
predicted_Qfwd_original = np.expm1(predicted_Qfwd_original) 
predicted_Qback_original = np.expm1(predicted_Qback_original)  

# Display the predicted Qfwd and Qback in original scale
print("Predicted Qfwd (Original Scale):", predicted_Qfwd_original)
print("Predicted Qback (Original Scale):", predicted_Qback_original)

# Add the predicted values (in original scale) to the DataFrame
best_df["predicted_Qfwd"] = predicted_Qfwd_original.tolist()
best_df["predicted_Qback"] = predicted_Qback_original.tolist()

I0000 00:00:1740667478.267922   34065 service.cc:146] XLA service 0x76ae24005920 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740667478.267943   34065 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-02-27 15:44:38.292197: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-27 15:44:38.387877: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


1/4 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step

I0000 00:00:1740667478.830066   34065 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 290ms/step
Predicted Qfwd (Original Scale): [[ 8.0722885   6.878783    5.8905993  ...  4.853406    4.450183
   4.2515593 ]
 [ 2.338696    2.159307    2.0634384  ...  2.3122644   2.2919102
   2.0171947 ]
 [ 1.4390088   1.7832248   1.8661842  ...  0.59331703  0.41394895
   0.19635247]
 ...
 [ 9.828346    8.294578    7.2111783  ...  1.7743905   1.7769928
   1.5042553 ]
 [13.243763   10.885319   10.234821   ...  4.6918955   4.3835716
   4.3053055 ]
 [ 8.263265    6.950822    6.2902827  ...  3.2840333   3.447416
   3.4464285 ]]
Predicted Qback (Original Scale): [[ 0.33133823  0.38059816  0.45955068 ...  5.88366     5.4498744
   5.3031816 ]
 [ 0.78923774  0.7292985   0.54657835 ...  2.7855022   2.5509613
   2.4260504 ]
 [ 1.8866854   1.9344102   1.5914586  ... -0.01422545  0.03148752
  -0.04805873]
 ...
 [ 0.56454337  0.7690506   0.9569939  ...  2.3465683   2.169724
   2.1041913 ]
 [ 0.63961995  0.685864    0.7626273  ...  4.180428    4.085953
   4.182763  ]
 [ 0.

In [25]:
best_df["log_Qfwd"] = best_df["mie_Qfwd"].apply(lambda x: np.log1p(np.array(x)))
best_df["log_Qback"] = best_df["mie_Qback"].apply(lambda x: np.log1p(np.array(x)))
best_df.head()

,mat_core,mat_shell,r_core,r_shell,Sample Index,Runtime (seconds),Loss,mie_Qfwd,mie_Qback,predicted_Qfwd,predicted_Qback,log_Qfwd,log_Qback
0,ZrO2,Au,77.337858,103.499478,1,4.338133,0.130269,"[7.943693523704477, 7.631676005310999, 7.35065...","[0.12938060934056017, 0.20866880941269564, 0.2...","[8.072288513183594, 6.878783226013184, 5.89059...","[0.33133822679519653, 0.38059815764427185, 0.4...","[2.1909486496924258, 2.1554386931122766, 2.122...","[0.1216693491173211, 0.1895195964809834, 0.262..."
1,SiO2,Au,65.444002,84.980111,2,4.323697,0.005362,"[2.6381263938781765, 2.5292054290052843, 2.427...","[0.33209261045765476, 0.3435998575540038, 0.35...","[2.338696002960205, 2.1593070030212402, 2.0634...","[0.7892377376556396, 0.72929847240448, 0.54657...","[1.291468822174709, 1.261072754675092, 1.23169...","[0.2867510970718935, 0.29535247273783766, 0.30..."
2,Si,Si,48.419073,31.936499,3,3.205805,0.000001,"[1.137715927515656, 0.8148088614955376, 0.6078...","[0.11457456575332535, 0.03793865784512998, 0.0...","[1.4390088319778442, 1.7832248210906982, 1.866...","[1.8866853713989258, 1.9344102144241333, 1.591...","[0.7597379354475057, 0.5959801517004989, 0.474...","[0.10847277672312375, 0.037236686509116546, 0...."
3,ZrO2,Si3N4,93.356095,117.463630,4,4.147974,0.002124,"[20.180305365698967, 17.00894306872726, 14.976...","[1.502571295840774, 1.187915005201223, 1.03181...","[15.712120056152344, 13.141104698181152, 12.28...","[1.2991472482681274, 1.1873573064804077, 1.035...","[3.053071757713282, 2.8908684716649673, 2.7711...","[0.9173187216478738, 0.7829490379191183, 0.708..."
4,Au,TiO2,49.023123,77.253263,5,4.145138,0.008713,"[6.220173089120092, 6.539505428696962, 6.85150...","[1.9077666986221749, 1.8125993548908415, 1.691...","[5.192962646484375, 5.916505813598633, 6.53437...","[2.2177658081054688, 2.3039534091949463, 2.199...","[1.9768789261797133, 2.020156586857536, 2.0607...","[1.0673853290189652, 1.0341090930899939, 0.990..."


## Save best geometries data 

In [26]:
best_geometries_path = "best_geometries/global_inverse_test_data.pkl"

# Saving DataFrames
with open(best_geometries_path, "wb") as f:
    pickle.dump(best_df, f)